# Test Notebook for Trying Stuff Out #

### I. Webscraping Tracker.gg Test 1 ###

In [28]:
# Import dependencies
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import urllib.parse as urlparse


In [21]:
# Initialize data frame to store data
player_data = pd.DataFrame()

In [29]:
# Load undetected Chrome driver
options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = uc.Chrome(options=options)

In [23]:
MAX_PLAYER_PER_REGION = 50
PLAYER_NAME_DF_OUTPUT = f"top_{MAX_PLAYER_PER_REGION}_players_per_region_4_6.csv" # today is 4/6/25 

In [24]:
# Scrape player usernames
leaderboard_urls = [
    "https://tracker.gg/valorant/leaderboards/ranked/all/default?platform=pc&region=na&act=dcde7346-4085-de4f-c463-2489ed47983b&page=1",
    "https://tracker.gg/valorant/leaderboards/ranked/all/default?platform=pc&region=eu&act=dcde7346-4085-de4f-c463-2489ed47983b&page=1",
    "https://tracker.gg/valorant/leaderboards/ranked/all/default?platform=pc&region=ap&act=dcde7346-4085-de4f-c463-2489ed47983b&page=1",
    "https://tracker.gg/valorant/leaderboards/ranked/all/default?platform=pc&region=kr&act=dcde7346-4085-de4f-c463-2489ed47983b&page=1",
    "https://tracker.gg/valorant/leaderboards/ranked/all/default?platform=pc&region=br&act=dcde7346-4085-de4f-c463-2489ed47983b&page=1",
    "https://tracker.gg/valorant/leaderboards/ranked/all/default?platform=pc&region=latam&act=dcde7346-4085-de4f-c463-2489ed47983b&page=1"
]

players = []

for url in leaderboard_urls:
    driver.get(url)
    print(f"Loading: {url}")
    region = urlparse.parse_qs(urlparse.urlparse(url).query).get("region", ["unknown"])[0]
    
    try:
        # Wait for the first v3-trnign span to load
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "v3-trnign"))
        )

        # Find all player blocks
        player_blocks = driver.find_elements(By.XPATH, '//div[contains(@class, "flex gap-2 items-center")]')

        count = 0
        for block in player_blocks:
            if count >= MAX_PLAYER_PER_REGION:
                break

            try:
                parent_span = block.find_element(By.CLASS_NAME, "v3-trnign")
                username = parent_span.find_element(By.CLASS_NAME, "truncate").text.strip()
                tag = parent_span.find_element(By.CLASS_NAME, "v3-trnign__discriminator").text.strip().replace("#", "")

                if username and tag:
                    players.append({"username": username, "tag": tag, "region": region})
                    count += 1

            except Exception as e:
                print("Skipping block (likely non-player):", e)
                continue

    except Exception as e:
        print("Error loading page:", e)

print(f"✅ Total players scraped: {len(players)}")

Loading: https://tracker.gg/valorant/leaderboards/ranked/all/default?platform=pc&region=na&act=dcde7346-4085-de4f-c463-2489ed47983b&page=1
Skipping block (likely non-player): Message: no such element: Unable to locate element: {"method":"css selector","selector":".v3-trnign"}
  (Session info: chrome=135.0.7049.42); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x01075413+60691]
	GetHandleVerifier [0x01075454+60756]
	(No symbol) [0x00EA0693]
	(No symbol) [0x00EE8660]
	(No symbol) [0x00EE89FB]
	(No symbol) [0x00EDDDA1]
	(No symbol) [0x00F0D0A4]
	(No symbol) [0x00EDDCC4]
	(No symbol) [0x00F0D2D4]
	(No symbol) [0x00F2E6FB]
	(No symbol) [0x00F0CE56]
	(No symbol) [0x00EDC5F3]
	(No symbol) [0x00EDD444]
	GetHandleVerifier [0x012BBBC3+2446531]
	GetHandleVerifier [0x012B7198+2427544]
	GetHandleVerifier [0x012D2D5E+2541150]
	GetHandleVerifier [0x0108C3C5+154821]
	GetH

In [25]:
player_df = pd.DataFrame(players)
player_df

,username,tag,region
0,SEN nightz,aim,na
1,the chosen one,mØØda,na
2,humblegod,1122,na
3,ENVY POPPIN,GUIDE,na
4,KRÜ Shyy,Flow,na
...,...,...,...
295,javierjjvhz96,jvhz,latam
296,Hetzen,1MORE,latam
297,AUS 21,2121,latam
298,DG c0ldd,616,latam


In [26]:
player_df.to_csv(PLAYER_NAME_DF_OUTPUT, index=False)

In [30]:
# Test username and tag (mine)
username = "GV Stimulated"
safe_username = username.replace(" ", "_")
tag = "STIMD"
print(safe_username)
print(tag)

GV_Stimulated
STIMD


In [31]:
# Go to tracker page for EP9:A3 
url = f"https://tracker.gg/valorant/profile/riot/{safe_username}%23{tag}/overview?platform=pc&playlist=competitive&season=dcde7346-4085-de4f-c463-2489ed47983b"
print(url)
driver.get(url)

https://tracker.gg/valorant/profile/riot/GV_Stimulated%23STIMD/overview?platform=pc&playlist=competitive&season=dcde7346-4085-de4f-c463-2489ed47983b


In [ ]:
stats = {}
# Get RR value
try:
    # Find all stat blocks
    stat_blocks = driver.find_elements(By.CLASS_NAME, "stat")

    for block in stat_blocks:
        try:
            sup = block.find_element(By.TAG_NAME, "sup").text.strip()
            if sup == "RR":
                # rank_text = block.find_element(By.CLASS_NAME, "stat__label").text.strip()   # e.g., Immortal 2
                rr_value = block.find_element(By.CLASS_NAME, "stat__value").text.strip().replace("RR", "").strip()  # remove the RR

                # stats["rank_tier"] = rank_text
                stats["rank_rating"] = rr_value
                break  # found it!
        except:
            continue
except Exception as e:
    print("Couldn't find rank info:", e)

In [ ]:
stats["rank_rating"]

In [ ]:
# Wait until the "Show More" button is clickable and click it
try:
    show_more_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Show More")]'))
    )
    ActionChains(driver).move_to_element(show_more_button).click().perform()
    print("Clicked Show More!")
except Exception as e:
    print("Couldn't find or click the Show More button:", e)

In [ ]:
import re

def clean_label(label):
    # Strip, remove spaces around slashes, then replace remaining spaces with _
    label = label.strip()
    label = re.sub(r"\s*/\s*", "/", label)         # Turn " / " into "/"
    label = re.sub(r"\s+", "_", label)             # Turn all spaces into underscores
    label = re.sub(r"[^\w/]+", "_", label)         # Replace any other symbols (like : or ,) with _
    label = label.strip("_")                       # Remove leading/trailing underscores
    return label.lower()                           # Optional: lowercase everything

In [ ]:
# Step 1: Find all stat sections (Attack, Defense, Clutch, etc.)
stat_sections = driver.find_elements(By.XPATH, '//div[contains(@class, "bg-surface-1 flex flex-col gap-px")]')

for section in stat_sections:
    try:
        # Step 2: Get the category title (e.g., "Attack", "Defense", etc.)
        header = section.find_element(By.XPATH, './/div[contains(@class, "font-medium text-20")]')
        category = header.text.strip().replace(" ", "_")  # e.g., "Attack" → "Attack", or "Clutch Performance" → "Clutch_Performance"
    except:
        category = "General"

    # Step 3: Get all stat blocks within the current section
    stat_blocks = section.find_elements(By.XPATH, './/div[contains(@class, "stat flex")]')

    for block in stat_blocks:
        try:
            label = block.find_element(By.CLASS_NAME, "font-normal").text.strip()
            value = block.find_element(By.CLASS_NAME, "font-medium").text.strip()
            # Prefix the stat label with the category
            label_clean = clean_label(label)
            full_label = f"{category}_{label_clean}"
            stats[full_label] = value
        except Exception as e:
            print(f"Skipping stat in {category}: {e}")

# Show the results
test_data = pd.DataFrame([stats])

# Add player's data to dataframe
if stats:
    player_data = pd.concat([player_data, pd.DataFrame([stats])], ignore_index=True)

In [ ]:
test_data

In [32]:
driver.quit()

### II. Webscraping Tracker.GG Troubleshooting ###

**Issue:** scrape.py is able to scrape some players but results in a lot of missing values (either entire players missing or partially collected data within each player). Need to make the scrape.py script more robust.

In [8]:
from scrape import *

In [2]:
player_df = pd.read_csv("top_50_players_per_region.csv")

In [3]:
# Load undetected Chrome driver
options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")
driver = uc.Chrome(options=options)

2025-04-06 09:37:53,358 [INFO] patching driver executable C:\Users\Polar\appdata\roaming\undetected_chromedriver\undetected_chromedriver.exe


In [ ]:
# # Show issue
# sample_data_1 = pd.read_csv("scraped_player_stats_2.csv")
# sample_data_1.head()

In [4]:
nightz = player_df.iloc[0]
print(nightz)

username    SEN nightz
tag                aim
region              na
Name: 0, dtype: object


In [5]:
test_scrape = scrape(nightz.username, nightz.tag, nightz.region, driver)

2025-04-06 09:38:25,445 [INFO] Clicked Show More!


In [6]:
test_scrape

{'IGN': 'SEN nightz#aim',
 'region': 'na',
 'rank_rating': '1,126',
 'Combat_ability_1_casts': '6,102',
 'Combat_ability_1_casts/match': '15.07',
 'Combat_ability_1_casts/round': '0.7',
 'Combat_ability_2_casts': '11,629',
 'Combat_ability_2_casts/match': '28.71',
 'Combat_ability_2_casts/round': '1.4',
 'Combat_aces': '24',
 'Combat_acs': '263.6',
 'Combat_assists': '2,140',
 'Combat_assists/match': '5.3',
 'Combat_assists/round': '0.3',
 'Combat_clutch': '1.9%',
 'Combat_clutches': '164',
 'Combat_clutches__1v1': '92',
 'Combat_clutches__1v2': '52',
 'Combat_clutches__1v3': '13',
 'Combat_clutches__1v4': '6',
 'Combat_clutches_lost__1v2': '141',
 'Combat_clutches_lost__1v3': '190',
 'Combat_clutches_lost__1v4': '183',
 'Combat_clutches_lost__1v5': '97',
 'Combat_damage': '1,465,563',
 'Combat_damage_delta_δ': '251,342',
 'Combat_damage_received': '1,214,221',
 'Combat_damage/match': '3,618.67',
 'Combat_damage/minute': '116.7',
 'Combat_damage/round': '172.2',
 'Combat_ddδ/round': '3

In [7]:
driver.close()